# Error handling

Error handling in Rust is very different from both C and C++. The Rust's approach to error handling has advantages and downsides to both, and it worth a chapter to cover.

## Error handling in C

C doesn't have exceptions, try/catch and stack unwinding. It uses return values for declaring that something bad happened, and global variables if more information is needed. The similar is possible is Rust, but we (and compiler) don't like global mutable variables since they are not thread safe. Even if they were thread safe (we can use thread local variables for example), since global variables are ugly, and Rust has ADT, we can use an enum to store error data inside the return value:

In [3]:
enum ReturnTypeOfAFunction {
    EverythingIsFine {
        result: i32,
    },
    ConnectionFailure {
        url: String,
    },
    DiskError {
        file_name: String,
    },
    UnknownError,
}

This pattern is not specific to this function, so the standard library has `enum Result<T, E>` with variants `Ok(T)` and `Err(E)`. We can make the above type using it:

In [ ]:
enum ErrorsOfAFunction {
    ConnectionFailure {
        url: String,
    },
    DiskError {
        file_name: String,
    },
    UnknownError,
}

// type alias
type ReturnTypeOfAFunction = Result<i32, ErrorsOfAFunction>;

The Rust standard library and ecosystem widely use the `Result` enum for error handling:

In [5]:
use std::fs;

fs::read_to_string("non-existent.txt")

Err(Os { code: 2, kind: NotFound, message: "No such file or directory" })

With `Result`, try/catch is just a `match`:

In [14]:
let open_result = fs::read_to_string("non-existent.txt");
match open_result {
    Ok(file_string) => {
        println!("File content is: {}", file_string);
    }
    Err(error) => {
        println!("Some error happened: {}", error);
    }
}
println!("Other codes");

Some error happened: No such file or directory (os error 2)
Other codes


## ? Operator

Now think we are in a function, and want to cascade error. We can use this code:

In [18]:
fn find_count_of_a_in_a_file(path: &str) -> Result<usize, std::io::Error> {
    let open_result = fs::read_to_string("non-existent.txt");
    let text = match open_result {
        Ok(file_string) => file_string,
        Err(error) => return Err(error),
    };
    let result = text.chars().filter(|x| *x == 'a').count();
    Ok(result)
}

find_count_of_a_in_a_file("non-existent.txt")

Err(Os { code: 2, kind: NotFound, message: "No such file or directory" })

Since cascading error is a too common thing, Rust has an operator for it, `?` operator. `expr?` means match `expr` and in case of `Err`, return the error from current function, and in case of `Ok`, evaluate `expr?` to the inner thing of `Ok`. Example:

In [19]:
fn find_count_of_a_in_a_file(path: &str) -> Result<usize, std::io::Error> {
    let text = fs::read_to_string("non-existent.txt")?;
                                                // --^-- notice this
    let result = text.chars().filter(|x| *x == 'a').count();
    Ok(result)
}

find_count_of_a_in_a_file("non-existent.txt")

Err(Os { code: 2, kind: NotFound, message: "No such file or directory" })

You can't use `?` on results in functions that doesn't return result:

In [20]:
fn find_count_of_a_in_a_file_wrong(path: &str) -> usize {
    let text = fs::read_to_string("non-existent.txt")?;
    let result = text.chars().filter(|x| *x == 'a').count();
    result
}

Error: the `?` operator can only be used in a function that returns `Result` or `Option` (or another type that implements `FromResidual`)

Compiler spoiled the story. As compiler said, `?` also works in functions that return `Option<T>`, for things with type `Option<U>`:

In [21]:
fn third_from_an_optional_vector(input_opt: Option<Vec<i32>>) -> Option<i32> {
    let input = input_opt?; // return None if input was None
    let third_opt = input.get(2); // Similar to input[2], but returns Option
    let third = third_opt?; // Return None if there were no third element. You could do it in one line.
    Some(*third)
}

println!("{:?}", third_from_an_optional_vector(None));
println!("{:?}", third_from_an_optional_vector(Some(vec![1, 2, 3, 4])));
println!("{:?}", third_from_an_optional_vector(Some(vec![1])));

None
Some(3)
None


You can convert a result to an option with `.ok()`

In [22]:
let x: Result<i32, String> = Ok(12);
x.ok()

Some(12)

And an option to a result with `.ok_or`:

In [24]:
let x: Option<i32> = None;
x.ok_or("some_error")

Err("some_error")

So we can use `?` for `Option` in a function that returns `Result` and wise versa.

In [28]:
use std::io::Error;

#[derive(Debug)]
enum MyError {
    IoError(Error),
    PathNotProvided
}

impl From<Error> for MyError {
    fn from(e: Error) -> MyError {
        MyError::IoError(e)
    }
}

fn find_count_of_a_in_a_file(path: Option<&str>) -> Result<usize, MyError> {
    let path = path.ok_or(MyError::PathNotProvided)?;
    let text = fs::read_to_string("non-existent.txt")?;
    let result = text.chars().filter(|x| *x == 'a').count();
    Ok(result)
}

find_count_of_a_in_a_file(None)

Err(PathNotProvided)

As you can see, we can use `?` for different error types if they implement `From` trait.

## Control flow enum

There is another enum which supports `?` operator: `ControlFlow<B, C = ()>` with variants `Break(B)` and `Continue(C)`. For example, have this code:

In [29]:
for x in 1..20 {
    if x % 2 == 0 {
        println!("{x} 2");
        if x % 3 == 0 {
            println!("{x} 3");
            if x % 4 == 0 {
                println!("{x} 4");
                break;
            }
        }
    }
}

2 2
4 2
6 2
6 3
8 2
10 2
12 2
12 3
12 4


()

Now we want to refactor two inner ifs as a separate function. It doesn't work naively:

In [32]:
fn refactored(x: i32) {
    println!("{x} 2");
    if x % 3 == 0 {
        println!("{x} 3");
        if x % 4 == 0 {
            println!("{x} 4");
            break;
        }
    }
}

for x in 1..20 {
    if x % 2 == 0 {
        refactored(x);
    }
}

Error: `break` outside of a loop

We can use `ControlFlow` enum as a glorified `bool`:

In [33]:
use std::ops::ControlFlow;

fn refactored(x: i32) -> ControlFlow<()> {
    println!("{x} 2");
    if x % 3 == 0 {
        println!("{x} 3");
        if x % 4 == 0 {
            println!("{x} 4");
            return ControlFlow::Break(());
        }
    }
    ControlFlow::Continue(())
}

for x in 1..20 {
    if x % 2 == 0 {
        if refactored(x).is_break() {
            break;
        }
    }
}

2 2
4 2
6 2
6 3
8 2
10 2
12 2
12 3
12 4


()

Now Imagine we want to refactor it one more level. Without `?` it would look like this:

In [34]:
fn refactored2(x: i32) -> ControlFlow<()> {
    println!("{x} 3");
    if x % 4 == 0 {
        println!("{x} 4");
        return ControlFlow::Break(());
    }
    ControlFlow::Continue(())
}

fn refactored(x: i32) -> ControlFlow<()> {
    println!("{x} 2");
    if x % 3 == 0 {
        if refactored2(x).is_break() {
            return ControlFlow::Break(());
        }
    }
    ControlFlow::Continue(())
}

But with `?`, second refactoring becomes almost copy pasting the code into a new function:

In [ ]:
fn refactored2(x: i32) -> ControlFlow<()> {
    println!("{x} 3");
    if x % 4 == 0 {
        println!("{x} 4");
        return ControlFlow::Break(());
    }
    ControlFlow::Continue(())
}

fn refactored(x: i32) -> ControlFlow<()> {
    println!("{x} 2");
    if x % 3 == 0 {
        refactored2(x)?;
    }
    ControlFlow::Continue(())
}

Another usage of `ControlFlow` enum is in traversing trees (and graphs and anything). Let's define a simple tree data structure:

In [36]:
#[derive(Debug)]
enum Tree<T> {
    Leaf(T),
    Node(Vec<Tree<T>>), // vector of children
}

use Tree::*;

let example = Node(vec![Leaf(10), Node(vec![Leaf(20), Leaf(30), Leaf(40)]), Leaf(50)]);
println!("{:#?}", example);

Node(
    [
        Leaf(
            10,
        ),
        Node(
            [
                Leaf(
                    20,
                ),
                Leaf(
                    30,
                ),
                Leaf(
                    40,
                ),
            ],
        ),
        Leaf(
            50,
        ),
    ],
)


Now we want to find left most node which is bigger than a number:

In [43]:
fn left_most_bigger<T: Ord + Copy>(tree: &Tree<T>, than_this: T) -> Option<T> {
    match tree {
        Leaf(x) => (x > &than_this).then_some(*x),
        Node(children) => {
            for child in children {
                match left_most_bigger(&child, than_this) {
                    Some(x) => return Some(x),
                    None => continue,
                }
            }
            None
        }
    }
}

left_most_bigger(&example, 35)

Some(40)

Now let's change the `Option<T>` to `ControlFlow<T>`:

In [46]:
fn left_most_bigger<T: Ord + Copy>(tree: &Tree<T>, than_this: T) -> ControlFlow<T> {
    match tree {
        Leaf(x) if x > &than_this => ControlFlow::Break(*x),
        Leaf(x) => ControlFlow::Continue(()),
        Node(children) => {
            for child in children {
                left_most_bigger(&child, than_this)?;
            }
            ControlFlow::Continue(())
        }
    }
}

left_most_bigger(&example, 35)

Break(40)

We can use `?` and break from the outer function if we founded what we need. `ControlFlow` has reverse `?` semantic to `Option` and `Result`, and it will return in case of success. `break` with value is also supported in `loop`: 

In [47]:
let mut counter = 0;

let result = loop {
    counter += 1;

    if counter == 10 {
        break counter * 2;
    }
};

result

20

That's enough about control flow. Let's get back to error handling.

Error handling with `Result` has these benefits:
* Possible errors (or lack of error) of a function is clear from it's return type.
* Error handling is explicit. You can't forget that a function may throw error. You need to cascade it yourself with `?` explicitly.
* No need to support runtime for stack unwinding. This is an issue in low level system programming, like in OS or embedded systems. People using C++ in those use-cases use `-fno-exception`, but it will make any error in standard library and third party libraries impossible to catch and handle.

## Panic

There is another form of errors that might happen in a rust program:

In [48]:
:preserve_vars_on_panic 1

Preserve vars on panic: true


In [49]:
let v = vec![1, 2, 3];
v[5]

thread '<unnamed>' panicked at 'index out of bounds: the len is 3 but the index is 5', src/lib.rs:213:40
stack backtrace:
   0: rust_begin_unwind
             at /rustc/4b91a6ea7258a947e59c6522cd5898e7c0a6a88f/library/std/src/panicking.rs:584:5
   1: core::panicking::panic_fmt
             at /rustc/4b91a6ea7258a947e59c6522cd5898e7c0a6a88f/library/core/src/panicking.rs:142:14
   2: core::panicking::panic_bounds_check
             at /rustc/4b91a6ea7258a947e59c6522cd5898e7c0a6a88f/library/core/src/panicking.rs:84:5
   3: <core::panic::unwind_safe::AssertUnwindSafe<F> as core::ops::function::FnOnce<()>>::call_once
   4: run_user_code_24
   5: evcxr::runtime::Runtime::run_loop
   6: evcxr::runtime::runtime_hook
   7: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


Panic occurred, the following variables have been lost: v